<a href="https://colab.research.google.com/github/EiPhyuSinn/Retrieval-QA-Langchain-with-togetherAI-API/blob/main/qa_togetherai_mixtral_langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade langchain langchain-community


In [ ]:
pip install -U langchain-openai


In [3]:
from langchain_openai import ChatOpenAI
import os
from langchain_core.messages import HumanMessage

os.environ["TOGETHER_API_KEY"] = "your_api_key"

llm = ChatOpenAI(
    model="mistralai/Mixtral-8x7B-Instruct-v0.1",
    temperature=0.7,
    openai_api_key=os.environ["TOGETHER_API_KEY"],
    openai_api_base="https://api.together.xyz/v1",
)

response = llm.invoke([HumanMessage(content="What's the difference between AI and machine learning?")])
print(response.content)

 Artificial Intelligence (AI) is a broad field that refers to the development of computer systems that can perform tasks that typically require human intelligence, such as visual perception, speech recognition, decision-making, and language translation.

Machine Learning (ML) is a subset of AI that focuses on the development of algorithms and statistical models that enable machines to learn and improve from data, without being explicitly programmed. In other words, machine learning is a technique for realizing AI.

So, while AI is the broader concept of machines being able to carry out tasks that would normally require human intelligence, machine learning is a specific approach to achieving AI by using data and algorithms to enable machines to learn and improve over time.

Here are some key differences between AI and machine learning:

1. Scope: AI is a broader concept that encompasses various techniques and approaches to achieve intelligence in machines, while machine learning is a sp

In [ ]:
! pip install sentence-transformers


In [6]:
from langchain.embeddings import HuggingFaceEmbeddings

embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vector = embedding.embed_query("Hello, how are you?")


<ipython-input-6-686dae6903e9>:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [7]:
len(vector)

384

In [8]:
from google.colab import files
uploaded = files.upload()


Saving Mental_Health_FAQ.csv to Mental_Health_FAQ.csv


In [22]:
from langchain.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(file_path='Mental_Health_FAQ.csv',source_column='Questions')
data = loader.load()
data[:3]

[Document(metadata={'source': 'What does it mean to have a mental illness?', 'row': 0}, page_content='Question_ID: 1590140\nQuestions: What does it mean to have a mental illness?\nAnswers: Mental illnesses are health conditions that disrupt a personâ€™s thoughts, emotions, relationships, and daily functioning. They are associated with distress and diminished capacity to engage in the ordinary activities of daily life.\nMental illnesses fall along a continuum of severity: some are fairly mild and only interfere with some aspects of life, such as certain phobias. On the other end of the spectrum lie serious mental illnesses, which result in major functional impairment and interference with daily life. These include such disorders as major depression, schizophrenia, and bipolar disorder, and may require that the person receives care in a hospital.\nIt is important to know that mental illnesses are medical conditions that have nothing to do with a personâ€™s character, intelligence, or wil

In [10]:
pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 67.0 MB/s eta 0:00:00


In [11]:
from langchain_community.vectorstores import FAISS

db = FAISS.from_documents(documents=data,
                          embedding=embedding)
retriever = db.as_retriever(score_threshold = 0.7)
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7e27a35bb010>, search_kwargs={})

In [12]:
from langchain.prompts import PromptTemplate

custom_prompt = """
You are provided with some context and a question. Answer the question **only** using the given context.
Focus on extracting as much detail as possible from the "response" section in the document.
If the context does not contain the answer, simply reply with "I don't know." Do not make up answers.

Context:
{context}

Question:
{question}
"""

PROMPT = PromptTemplate(
    template=custom_prompt,
    input_variables=["context", "question"]
)

chain_type_kwargs = {"prompt": PROMPT}

from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    input_key="query",
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs
)


In [14]:
answer1 = qa_chain("What does it mean to have a mental illness?")

In [16]:
answer1.keys()

dict_keys(['query', 'result', 'source_documents'])

In [18]:
answer1['query']

'What does it mean to have a mental illness?'

In [19]:
answer1['result']

" To have a mental illness means to have a health condition that disrupts a person's thoughts, emotions, relationships, and daily functioning. These conditions are associated with distress and diminished capacity to engage in the ordinary activities of daily life. Mental illnesses fall along a continuum of severity, from mild conditions that only interfere with some aspects of life, to serious mental illnesses that result in major functional impairment and interference with daily life. Examples of serious mental illnesses include major depression, schizophrenia, and bipolar disorder, and may require hospitalization. It is important to note that mental illnesses are medical conditions due to the brain's biology and have nothing to do with a person's character, intelligence, or willpower. With proper treatment, which often includes a combination of medication and social support, 70-90 percent of individuals receiving treatment experience a reduction in symptoms and an improved quality of

In [20]:
answer1['source_documents']

[Document(id='c3159ae1-d53c-4179-a723-033b462369e1', metadata={'source': 'What does it mean to have a mental illness?', 'row': 0}, page_content='Question_ID: 1590140\nQuestions: What does it mean to have a mental illness?\nAnswers: Mental illnesses are health conditions that disrupt a personâ€™s thoughts, emotions, relationships, and daily functioning. They are associated with distress and diminished capacity to engage in the ordinary activities of daily life.\nMental illnesses fall along a continuum of severity: some are fairly mild and only interfere with some aspects of life, such as certain phobias. On the other end of the spectrum lie serious mental illnesses, which result in major functional impairment and interference with daily life. These include such disorders as major depression, schizophrenia, and bipolar disorder, and may require that the person receives care in a hospital.\nIt is important to know that mental illnesses are medical conditions that have nothing to do with a